Webscrapping & Data Processing <br>
KARUNATHASAN Nilany <br>
SAMBATH Sïndoumady <br>
[Github Link](https://github.com/nilanyK/Webscrapping-DataProcessing)<br>
[Streamlit App Link](https://webscrapping-dataprocessing-cezgc8agfx5pjs29ct9yvg.streamlit.app/)<br>
DIA2

#### <center><font  align="center"> Project : Sustainable Nike Sneaker Marketplace 👟</font> <center>
# <center><font  align="center"> Vestaire Collective Scrapping </font> <center>
<div style="text-align:center;">
    <span style="display:inline-block; margin-right: 20px;">
        <img src="https://getlogovector.com/wp-content/uploads/2020/10/vestiaire-collective-logo-vector.png" alt="Trustpilot Logo" width="100"/>
    </span>
</div>


___

## SUMMARY

[1. Vestiaire CO Data Scraping and Processing](#1.-Vinted-Data-Scraping-and-Processing) <br>
[2. Approximate Carbon Footprint Calculation](#2.-Approximate-Carbon-Footprint-Calculation) <br>
[3. Nike Price Scraping for Profit Calculation](#3.-Nike-Price-Scraping-for-Profit-Calculation) <br>
[4. Top 5 Most Sold Sneakers Scrapping](#4.-Top-5-Most-Sold-Sneakers-Scrapping) <br>
[5. Final Concataned Dataframs VINTED & Vestiaire Collective](#5.-Final-Concataned-Dataframs-VINTED-&-Vestiaire-Collective)

## 1. Vestiaire CO Data Scraping and Processing

The following code presents the method for retrieving data from Vestiaire Collective, focusing on Nike brand shoes. The get_vestaire_data function is structured to adapt to user preferences by incorporating parameters such as gender, colour and size, allowing the search to be tailored to specific criteria. The function also manages the retrieval of the seller's location and the calculation of their carbon footprint in relation to Paris.

In [1]:
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
import pandas as pd
#Scrapping vestiaire co to retrieve data
def get_vestiaire_data(url, max_rows):
    vestiaire_url = url
    
    # Use Firefox Driver
    driver = webdriver.Firefox()
    driver.get(vestiaire_url)

    # Cookies handling
    try:
        cookie_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, 'popin_tc_privacy_button_2'))
        )
        cookie_button.click()
    except:
        print("Cookie button not found or timed out")

    data = {"Title": [], "Price": [], "Brand": [],  "Color" :[], "Size": [], "Link": [], "Image Source": [], "Localisation": []}
    
    counter = 0  # Initialize a counter to keep track of the number of rows scraped

    # Start to scrap articles
    while counter < max_rows:
        try:
            # To see the entire page
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CLASS_NAME, 'product-search_catalog__flexContainer__Dg0eL'))
            )

            #Point article container
            product_containers = driver.find_element(By.CLASS_NAME, 'product-search_catalog__flexContainer__Dg0eL')
            li_elements = product_containers.find_elements(By.TAG_NAME, 'li')
            
            #Iterateon each article
            for container in li_elements:
                try:
                    # Retreive each informations if exists
                    try:
                        title_element = container.find_element(By.XPATH, './/h2[@data-cy="productCard__productLink"]')
                    except NoSuchElementException:
                        continue

                    brand_element = title_element.find_element(By.XPATH, './/span[@data-cy="productCard__text__brand"]')
                    name_element = title_element.find_element(By.XPATH, './/span[@data-cy="productCard__text__name"]')
                    size_element = title_element.find_element(By.XPATH, './/p[@data-cy="productCard__text__size"]')
                    price_element = container.find_element(By.XPATH, './/span[@data-cy="productCard__text__price__discount"]')
                    location_element = container.find_element(By.XPATH, './/div[@data-cy="productCard__text__location"]')

                    try:
                        link_element = container.find_element(By.XPATH, './/a[@class="product-card_productCard__image__40WNk"]')
                        href = link_element.get_attribute('href')
                    except NoSuchElementException:
                        continue

                    img_element = container.find_element(By.XPATH, './/img[@class="vc-images_image__TfKYE"]')
                    img_src = img_element.get_attribute('src')

                    localisation = location_element.text.strip()

                    # Extract localisation information from article's href link
                    driver.execute_script("window.open('', '_blank');")
                    driver.switch_to.window(driver.window_handles[1])
                    driver.get(href)

                    try:
                       
                        color_element = WebDriverWait(driver, 10).until(
                       EC.presence_of_element_located((By.XPATH, '//li[@class="product-description-list_descriptionList__listItem__Dw4lT"]/span[@class="product-description-list_descriptionList__property__MPK2a"][text()="Couleur :"]/following-sibling::span[@class="product-description-list_descriptionList__value__lJeA_"]'))
                    )
                        color = color_element.text
                    except:
                        color = None

                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])

                    #build dataset
                    data["Title"].append(f"{brand_element.text.strip()} {name_element.text.strip()}")
                    data["Price"].append(price_element.text.strip())
                    data["Brand"].append(brand_element.text.strip())
                    data["Size"].append(size_element.text.strip())
                    data["Link"].append(href)
                    data["Image Source"].append(img_src)
                    data["Localisation"].append(localisation)
                    data["Color"].append(color)
                    counter += 1 
                except StaleElementReferenceException:
                    print("Stale Element Reference Exception - Skipping container.")
                    continue
                except ValueError as ve:
                    print(f"ValueError: {ve}")
                    continue
                except Exception as e:
                    print(f"An unexpected error occurred: {e}")
                    continue

                if counter >= max_rows:
                    break  # Exit the loop if the counter reaches or exceeds the desired number of rows
            
            #Go to the next page
            try:
                next_button = driver.find_element(By.XPATH, "//button[@data-cy='pagination-right-arrow-btn']")
                if next_button.get_attribute("disabled"):
                    break  # Break the loop if 'next' button is disabled (last page)
                next_button.click()
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                WebDriverWait(driver, 10).until(
                    EC.presence_of_all_elements_located((By.CLASS_NAME, 'product-search_catalog__flexContainer__Dg0eL'))
                )
            except NoSuchElementException:
                break  # Break the loop if 'next' button is not found (no more pages)
        
        except NoSuchElementException:
            print("Product containers not found on the current page.")
    
    return pd.DataFrame(data)

#Data pre-processing
def process_data_vestiaire(vestiaire_data):
    vestiaire_data["Price"] = vestiaire_data['Price'].apply(lambda x: min([float(val.replace(',', '.').replace('\u202f', '')) for val in str(x).split('€') if val.strip()]))
    vestiaire_data['Size'] = vestiaire_data['Size'].str.extract(r'Taille : (\d+\.?\d*)')
    return vestiaire_data
   



## 2. Approximate Carbon Footprint Calculation

After extracting the data, the analysis was expanded to incorporate the environmental impact of shipping items from their respective locations to Paris. To quantify this impact, a function was developed to calculate the approximate carbon footprint associated with the localization of each article.

In the analysis, a constant emission rate per kilometer was assigned, and the carbon footprint for each item's journey to Paris was calculated. Additionally, a fixed value for Nike's carbon footprint from ordering on their website was introduced, set at 405 kg CO2 per ton (explained in our report).
<br>
The "Carbon Profit" column in the DataFrame reflects the environmental benefit attained by ordering through Vinted. It takes into account the reduction in carbon footprint compared to ordering directly from Nike's website.

In [2]:
from geopy.distance import geodesic
from geopy.geocoders import Nominatim

# Initialize geocoder
geolocator = Nominatim(user_agent="vinted_carbon_footprint")

def calculate_approximate_carbon_footprint(location):
    try:
        # Geocode the location to get coordinates
        location_info = geolocator.geocode(location, timeout=10)

        if location_info:
            location_coordinates = (location_info.latitude, location_info.longitude)

            # Coordinates of Paris, France
            paris_coordinates = (48.8566, 2.3522)

            # Calculate the distance between the location and Paris using geopy
            distance_km = geodesic(location_coordinates, paris_coordinates).kilometers

            # Assume a constant emission rate per kilometer 
            emission_rate_per_km = 0.2  
            # Calculate the approximate carbon footprint
            carbon_footprint = distance_km * emission_rate_per_km

            return carbon_footprint

    except (ValueError, TypeError, GeocoderTimedOut):
        # Handle cases where geocoding fails or times out
        return None


## 3. Nike Price Scraping for Profit Calculation

To assess potential cost savings, a process was initiated to scrape all available prices for the specified query on the Nike website. This allows for the calculation of the average price of the desired item by aggregating the individual prices. 

In [3]:
from selenium import webdriver

from selenium.webdriver.common.by import By

def accept_cookies(driver):
    try:
        # Accept cookies
        accept_cookies_button = driver.find_element(By.CLASS_NAME, 'btn-primary-dark')
        accept_cookies_button.click()
    except:
        pass  

def get_nike_prices(url):
    driver = webdriver.Firefox()
    
    try:
        # Go to nike.fr and accept cookies
        driver.get("https://www.nike.com/fr/")
        accept_cookies(driver)

    
        # Navigate to the URL with the search query
        driver.get(url)

        # Wait for the prices to load 
        driver.implicitly_wait(10)

        # Retrieve prices
        price_elements = driver.find_elements(By.CLASS_NAME, 'product-price')
        prices = [float(price.text.replace('€', '').replace(',', '.')) for price in price_elements]

        # Calculate the average price
        average_price = sum(prices) / len(prices)

        return average_price

    except Exception as e:
        print("An error occurred:", e)

    finally:
        driver.quit()



A mechanism is implemented to calculate the potential profit made by purchasing items on Vinted compared to the average price of the same product on the Nike website. The function computes the profit made for each item. The resulting DataFrame, processed_data, is then enriched with additional columns, providing insights into the potential financial benefit of opting for Vinted over the average market price on Nike.

In [4]:
def calculate_profit_made(row, average_nike_price):
    try:
        item_price = float(row['Price'])
        profit_made = average_nike_price - item_price
        return profit_made

    except ValueError:
        return None

## 4. Top 5 Most Sold Sneakers Scrapping

### Nike Air Force 1

##### Femme

In [6]:
url = "https://fr.vestiairecollective.com/search/?q=nike+air+force+1#gender=Femme%231_categoryParent=Chaussures%233_condition=Jamais%20port%C3%A9%20avec%20%C3%A9tiquette%231-Jamais%20port%C3%A9%232-Tr%C3%A8s%20bon%20%C3%A9tat%233"
vestiaireco_data_airforce1_femme = get_vestiaire_data(url,1000)
df_nike_airforce1_Femme = process_data_vestiaire(vestiaireco_data_airforce1_femme)
df_nike_airforce1_Femme


Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping con

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation
0,NIKE Air Force 1 Baskets,52.50,NIKE,Blanc,39,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,France
1,NIKE Air Force 1 Baskets,85.05,NIKE,Multicolore,38,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,France
2,NIKE Air Force 1 Baskets,99.75,NIKE,Multicolore,39,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie
3,NIKE Air Force 1 Baskets,69.46,NIKE,Multicolore,40.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Pays-Bas
4,NIKE Air Force 1 Baskets,99.75,NIKE,Doré,38.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Espagne
...,...,...,...,...,...,...,...,...
727,NIKE Baskets Air Force 1 en cuir,114.48,NIKE,Multicolore,36,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Etats-Unis
728,NIKE Baskets,59.92,NIKE,Blanc,11.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Etats-Unis
729,NIKE Baskets en cuir,147.17,NIKE,Blanc,9,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Etats-Unis
730,NIKE Baskets en cuir,63.07,NIKE,Noir,5.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Etats-Unis


In [7]:
df_nike_airforce1_Femme["Gender"]="Women"
df_nike_airforce1_Femme

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender
0,NIKE Air Force 1 Baskets,52.50,NIKE,Blanc,39,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,France,Women
1,NIKE Air Force 1 Baskets,85.05,NIKE,Multicolore,38,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,France,Women
2,NIKE Air Force 1 Baskets,99.75,NIKE,Multicolore,39,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie,Women
3,NIKE Air Force 1 Baskets,69.46,NIKE,Multicolore,40.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Pays-Bas,Women
4,NIKE Air Force 1 Baskets,99.75,NIKE,Doré,38.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Espagne,Women
...,...,...,...,...,...,...,...,...,...
727,NIKE Baskets Air Force 1 en cuir,114.48,NIKE,Multicolore,36,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Etats-Unis,Women
728,NIKE Baskets,59.92,NIKE,Blanc,11.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Etats-Unis,Women
729,NIKE Baskets en cuir,147.17,NIKE,Blanc,9,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Etats-Unis,Women
730,NIKE Baskets en cuir,63.07,NIKE,Noir,5.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Etats-Unis,Women


In [8]:
df_nike_airforce1_Femme.to_csv("vc_data_NikeAirForce1_FEMME.csv", index=False)

##### Homme

In [9]:
url = "https://fr.vestiairecollective.com/search/?q=nike+air+force+1#gender=Homme%232_condition=Jamais%20port%C3%A9%20avec%20%C3%A9tiquette%231-Jamais%20port%C3%A9%232-Tr%C3%A8s%20bon%20%C3%A9tat%233_categoryParent=Chaussures%2313"
vestiaireco_data_airforce1_homme = get_vestiaire_data(url,900)
df_nike_airforce1_Homme = process_data_vestiaire(vestiaireco_data_airforce1_homme)
df_nike_airforce1_Homme


Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping con

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation
0,NIKE Air Force 1,73.50,NIKE,Blanc,39,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,France
1,NIKE Air Force 1,136.50,NIKE,Beige,42.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie
2,NIKE Air Force 1,39.90,NIKE,Vert,42.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie
3,NIKE Air Force 1,78.75,NIKE,Beige,10,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Croatie
4,NIKE Air Force 1,57.75,NIKE,Blanc,45.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie
...,...,...,...,...,...,...,...,...
537,NIKE Baskets basses Air Force 1,120.89,NIKE,Blanc,12,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Etats-Unis
538,NIKE Baskets basses Air Force 1,157.68,NIKE,Noir,10,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Etats-Unis
539,NIKE Baskets,62.02,NIKE,Autre,9.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Etats-Unis
540,NIKE Baskets basses en cuir,182.62,NIKE,Blanc,12,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Etats-Unis


In [10]:
df_nike_airforce1_Homme['Gender']="Men"
df_nike_airforce1_Homme

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender
0,NIKE Air Force 1,73.50,NIKE,Blanc,39,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,France,Men
1,NIKE Air Force 1,136.50,NIKE,Beige,42.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie,Men
2,NIKE Air Force 1,39.90,NIKE,Vert,42.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie,Men
3,NIKE Air Force 1,78.75,NIKE,Beige,10,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Croatie,Men
4,NIKE Air Force 1,57.75,NIKE,Blanc,45.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie,Men
...,...,...,...,...,...,...,...,...,...
537,NIKE Baskets basses Air Force 1,120.89,NIKE,Blanc,12,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Etats-Unis,Men
538,NIKE Baskets basses Air Force 1,157.68,NIKE,Noir,10,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Etats-Unis,Men
539,NIKE Baskets,62.02,NIKE,Autre,9.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Etats-Unis,Men
540,NIKE Baskets basses en cuir,182.62,NIKE,Blanc,12,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Etats-Unis,Men


In [11]:
df_nike_airforce1_Homme.to_csv("vc_data_NikeAirForce1_HOMME.csv",index=False)

##### Enfant

In [12]:
url = "https://fr.vestiairecollective.com/search/?q=nike+air+force+1#gender=Enfant%233_condition=Jamais%20port%C3%A9%20avec%20%C3%A9tiquette%231-Jamais%20port%C3%A9%232-Tr%C3%A8s%20bon%20%C3%A9tat%233_categoryParent=Chaussures%2311"
vestiaireco_data_airforce1_enfant = get_vestiaire_data(url,100)
df_nike_airforce1_Enfant = process_data_vestiaire(vestiaireco_data_airforce1_enfant)
df_nike_airforce1_Enfant


,Title,Price,Brand,Color,Size,Link,Image Source,Localisation
0,NIKE Air Force 1 Baskets,32.55,NIKE,Blanc,33,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,France
1,NIKE Air Force 1 Baskets,31.50,NIKE,Marron,36,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Allemagne
2,NIKE Air Force 1 Baskets,68.25,NIKE,Bleu,38,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,France
3,NIKE Air Force 1 Baskets,94.50,NIKE,Blanc,36,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,France
4,NIKE Air Force 1 en cuir Baskets,84.00,NIKE,Blanc,13.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie
...,...,...,...,...,...,...,...,...
95,NIKE Air Force 1 en cuir Baskets,65.17,NIKE,Noir,1,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Canada
96,NIKE Baskets Air Force 1 en toile,105.12,NIKE,Multicolore,7.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Etats-Unis
97,NIKE Baskets Air Force 1 en toile,73.58,NIKE,Gris,6,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Etats-Unis
98,NIKE Baskets Air Force 1 en cuir,81.57,NIKE,Bordeaux,6,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Royaume-Uni


In [13]:
df_nike_airforce1_Enfant['Gender']="Kid"
df_nike_airforce1_Enfant

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender
0,NIKE Air Force 1 Baskets,32.55,NIKE,Blanc,33,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,France,Kid
1,NIKE Air Force 1 Baskets,31.50,NIKE,Marron,36,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Allemagne,Kid
2,NIKE Air Force 1 Baskets,68.25,NIKE,Bleu,38,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,France,Kid
3,NIKE Air Force 1 Baskets,94.50,NIKE,Blanc,36,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,France,Kid
4,NIKE Air Force 1 en cuir Baskets,84.00,NIKE,Blanc,13.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie,Kid
...,...,...,...,...,...,...,...,...,...
95,NIKE Air Force 1 en cuir Baskets,65.17,NIKE,Noir,1,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Canada,Kid
96,NIKE Baskets Air Force 1 en toile,105.12,NIKE,Multicolore,7.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Etats-Unis,Kid
97,NIKE Baskets Air Force 1 en toile,73.58,NIKE,Gris,6,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Etats-Unis,Kid
98,NIKE Baskets Air Force 1 en cuir,81.57,NIKE,Bordeaux,6,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Royaume-Uni,Kid


In [14]:
df_nike_airforce1_Enfant.to_csv("vc_data_NikeAirForce1_ENFANT.csv",index=False)

##### Concataned Dataframe of Nike Air Force 1

In [15]:
df_Femme = pd.read_csv("vc_data_NikeAirForce1_FEMME.csv",sep=",")
df_Homme = pd.read_csv("vc_data_NikeAirForce1_HOMME.csv",sep=",")
df_Enfant = pd.read_csv("vc_data_NikeAirForce1_ENFANT.csv",sep=",")

In [16]:
df_nike_airforce1 = pd.concat([df_Femme, df_Homme, df_Enfant], axis=0, ignore_index=True)
df_nike_airforce1

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender
0,NIKE Air Force 1 Baskets,52.50,NIKE,Blanc,39.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,France,Women
1,NIKE Air Force 1 Baskets,85.05,NIKE,Multicolore,38.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,France,Women
2,NIKE Air Force 1 Baskets,99.75,NIKE,Multicolore,39.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie,Women
3,NIKE Air Force 1 Baskets,69.46,NIKE,Multicolore,40.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Pays-Bas,Women
4,NIKE Air Force 1 Baskets,99.75,NIKE,Doré,38.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Espagne,Women
...,...,...,...,...,...,...,...,...,...
1369,NIKE Air Force 1 en cuir Baskets,65.17,NIKE,Noir,1.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Canada,Kid
1370,NIKE Baskets Air Force 1 en toile,105.12,NIKE,Multicolore,7.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Etats-Unis,Kid
1371,NIKE Baskets Air Force 1 en toile,73.58,NIKE,Gris,6.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Etats-Unis,Kid
1372,NIKE Baskets Air Force 1 en cuir,81.57,NIKE,Bordeaux,6.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Royaume-Uni,Kid


In [17]:
# Adding columns to the DataFrame for the approximate carbon footprint and Nike's fixed value
df_nike_airforce1["Approximate Carbon Footprint (kg CO2)"] = df_nike_airforce1["Localisation"].apply(calculate_approximate_carbon_footprint)
df_nike_airforce1["Approximative Carbon Print from Ordering on Nike (kg CO2/ton)"] = 405  

# Adding a column for Carbon Profit
df_nike_airforce1["Carbon Profit (kg CO2)"] = df_nike_airforce1["Approximative Carbon Print from Ordering on Nike (kg CO2/ton)"] - df_nike_airforce1["Approximate Carbon Footprint (kg CO2)"]

df_nike_airforce1

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender,Approximate Carbon Footprint (kg CO2),Approximative Carbon Print from Ordering on Nike (kg CO2/ton),Carbon Profit (kg CO2)
0,NIKE Air Force 1 Baskets,52.50,NIKE,Blanc,39.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,France,Women,50.586158,405,354.413842
1,NIKE Air Force 1 Baskets,85.05,NIKE,Multicolore,38.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,France,Women,50.586158,405,354.413842
2,NIKE Air Force 1 Baskets,99.75,NIKE,Multicolore,39.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie,Women,211.591248,405,193.408752
3,NIKE Air Force 1 Baskets,69.46,NIKE,Multicolore,40.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Pays-Bas,Women,87.928759,405,317.071241
4,NIKE Air Force 1 Baskets,99.75,NIKE,Doré,38.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Espagne,Women,240.819687,405,164.180313
...,...,...,...,...,...,...,...,...,...,...,...,...
1369,NIKE Air Force 1 en cuir Baskets,65.17,NIKE,Noir,1.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Canada,Kid,1265.959945,405,-860.959945
1370,NIKE Baskets Air Force 1 en toile,105.12,NIKE,Multicolore,7.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Etats-Unis,Kid,1522.772797,405,-1117.772797
1371,NIKE Baskets Air Force 1 en toile,73.58,NIKE,Gris,6.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Etats-Unis,Kid,1522.772797,405,-1117.772797
1372,NIKE Baskets Air Force 1 en cuir,81.57,NIKE,Bordeaux,6.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Royaume-Uni,Kid,151.407889,405,253.592111


In [18]:
url1 = "https://www.nike.com/fr/w/air-force-1-chaussures-5sj3yzy7ok"
average_nike_price1=get_nike_prices(url1)
average_nike_price1

120.2961538461538

In [19]:
df_nike_airforce1.sort_values(by='Price', inplace=True)

# Calculate profit made for Vinted data
df_nike_airforce1['Average price of the product on Nike'] = average_nike_price1
df_nike_airforce1['Profit Made'] = df_nike_airforce1.apply(lambda row: calculate_profit_made(row, average_nike_price1), axis=1)

df_nike_airforce1

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender,Approximate Carbon Footprint (kg CO2),Approximative Carbon Print from Ordering on Nike (kg CO2/ton),Carbon Profit (kg CO2),Average price of the product on Nike,Profit Made
717,NIKE Baskets Air Force 1 en cuir,16.80,NIKE,Blanc,41.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,France,Women,50.586158,405,354.413842,120.296154,103.496154
1301,NIKE Air Force 1 en cuir Baskets,16.80,NIKE,Noir,25.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Irlande,Kid,170.405381,405,234.594619,120.296154,103.496154
708,NIKE Baskets Air Force 1,17.85,NIKE,Blanc,37.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Espagne,Women,240.819687,405,164.180313,120.296154,102.446154
1328,NIKE Baskets Air Force 1,19.95,NIKE,Rose,12.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Italie,Kid,211.591248,405,193.408752,120.296154,100.346154
1320,NIKE Baskets Air Force 1 en cuir,21.00,NIKE,Blanc,22.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,France,Kid,50.586158,405,354.413842,120.296154,99.296154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
702,NIKE Baskets en cuir,2890.77,NIKE,Multicolore,8.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Etats-Unis,Women,1522.772797,405,-1117.772797,120.296154,-2770.473846
1151,NIKE Baskets basses Air Force 1 en alligator,3889.40,NIKE,Marron,10.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Etats-Unis,Men,1522.772797,405,-1117.772797,120.296154,-3769.103846
1106,NIKE Air Force 1 en cuir,4680.23,NIKE,Vert,8.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Royaume-Uni,Men,151.407889,405,253.592111,120.296154,-4559.933846
1201,NIKE En cuir,15750.00,NIKE,Noir,10.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Italie,Men,211.591248,405,193.408752,120.296154,-15629.703846


In [20]:
df_nike_airforce1[df_nike_airforce1['Profit Made'].isna()]

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender,Approximate Carbon Footprint (kg CO2),Approximative Carbon Print from Ordering on Nike (kg CO2/ton),Carbon Profit (kg CO2),Average price of the product on Nike,Profit Made


In [21]:
df_nike_airforce1_cleaned= df_nike_airforce1.dropna(subset=['Profit Made'])

In [22]:
df_nike_airforce1_cleaned

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender,Approximate Carbon Footprint (kg CO2),Approximative Carbon Print from Ordering on Nike (kg CO2/ton),Carbon Profit (kg CO2),Average price of the product on Nike,Profit Made
717,NIKE Baskets Air Force 1 en cuir,16.80,NIKE,Blanc,41.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,France,Women,50.586158,405,354.413842,120.296154,103.496154
1301,NIKE Air Force 1 en cuir Baskets,16.80,NIKE,Noir,25.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Irlande,Kid,170.405381,405,234.594619,120.296154,103.496154
708,NIKE Baskets Air Force 1,17.85,NIKE,Blanc,37.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Espagne,Women,240.819687,405,164.180313,120.296154,102.446154
1328,NIKE Baskets Air Force 1,19.95,NIKE,Rose,12.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Italie,Kid,211.591248,405,193.408752,120.296154,100.346154
1320,NIKE Baskets Air Force 1 en cuir,21.00,NIKE,Blanc,22.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,France,Kid,50.586158,405,354.413842,120.296154,99.296154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
702,NIKE Baskets en cuir,2890.77,NIKE,Multicolore,8.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Etats-Unis,Women,1522.772797,405,-1117.772797,120.296154,-2770.473846
1151,NIKE Baskets basses Air Force 1 en alligator,3889.40,NIKE,Marron,10.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Etats-Unis,Men,1522.772797,405,-1117.772797,120.296154,-3769.103846
1106,NIKE Air Force 1 en cuir,4680.23,NIKE,Vert,8.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Royaume-Uni,Men,151.407889,405,253.592111,120.296154,-4559.933846
1201,NIKE En cuir,15750.00,NIKE,Noir,10.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Italie,Men,211.591248,405,193.408752,120.296154,-15629.703846


In [23]:
df_nike_airforce1_cleaned.to_csv("VC_AllData_Nike_AirForce1.csv",index=False)

### Nike Blazer

##### Femme

In [24]:
url = "https://fr.vestiairecollective.com/search/?q=nike+blazer#gender=Femme%231_categoryParent=Chaussures%233_condition=Jamais%20port%C3%A9%20avec%20%C3%A9tiquette%231-Tr%C3%A8s%20bon%20%C3%A9tat%233-Jamais%20port%C3%A9%232"
vestiaireco_data_airforce1_femme = get_vestiaire_data(url,300)
df_nike_blazer_Femme = process_data_vestiaire(vestiaireco_data_airforce1_femme)
df_nike_blazer_Femme

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation
0,NIKE Blazer Baskets,63.00,NIKE,Blanc,38,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie
1,NIKE Blazer Baskets,42.00,NIKE,Bordeaux,6,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Irlande
2,NIKE Blazer Baskets,63.00,NIKE,Blanc,41,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie
3,NIKE Blazer Baskets,55.65,NIKE,Vert,39,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie
4,NIKE Blazer en toile Baskets,42.00,NIKE,Noir,38,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie
...,...,...,...,...,...,...,...,...
295,NIKE Baskets en cuir,168.19,NIKE,Autre,11,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Etats-Unis
296,NIKE Blazer en cuir Baskets,71.40,NIKE,Blanc,38.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie
297,NIKE Blazer Baskets,97.65,NIKE,Blanc,39,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Espagne
298,NIKE Baskets Blazer en cuir,36.75,NIKE,Blanc,41,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Suède


In [25]:
df_nike_blazer_Femme['Gender']="Women"

In [26]:
df_nike_blazer_Femme.to_csv("vc_data_NikeBlazer_FEMME.csv",index=False)

##### Homme

In [27]:
url = "https://fr.vestiairecollective.com/search/?q=nike+blazer#gender=Homme%232_condition=Jamais%20port%C3%A9%20avec%20%C3%A9tiquette%231-Tr%C3%A8s%20bon%20%C3%A9tat%233-Jamais%20port%C3%A9%232_categoryParent=Chaussures%2313"
vestiaireco_data_airforce1_homme = get_vestiaire_data(url,290)
df_nike_blazer_Homme= process_data_vestiaire(vestiaireco_data_airforce1_homme)
df_nike_blazer_Homme

Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping con

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation
0,NIKE Blazer,82.95,NIKE,Blanc,42.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Allemagne
1,NIKE Blazer,78.75,NIKE,Rose,42.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Allemagne
2,NIKE Blazer,89.25,NIKE,Blanc,45,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Espagne
3,NIKE Blazer,78.75,NIKE,Blanc,43,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Espagne
4,NIKE Blazer,48.30,NIKE,Bleu,40,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,France
...,...,...,...,...,...,...,...,...
235,NIKE Baskets montantes en cuir,514.50,NIKE,Rouge,10.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Italie
236,NIKE Baskets montantes en cuir,514.50,NIKE,Multicolore,12,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Italie
237,NIKE Baskets montantes en cuir,514.50,NIKE,Multicolore,7.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Italie
238,NIKE Baskets montantes en cuir,514.50,NIKE,Bleu,7.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Italie


In [28]:
df_nike_blazer_Homme['Gender']="Men"

In [29]:
df_nike_blazer_Homme.to_csv("vc_data_NikeBlazer_HOMME.csv",index=False)

##### Enfant

In [30]:
url = "https://fr.vestiairecollective.com/search/?q=nike+blazer#condition=Jamais%20port%C3%A9%20avec%20%C3%A9tiquette%231-Tr%C3%A8s%20bon%20%C3%A9tat%233-Jamais%20port%C3%A9%232_gender=Enfant%233_categoryParent=Chaussures%2311"
vestiaireco_data_airforce1_enfant= get_vestiaire_data(url,18)
df_nike_blazer_Enfant= process_data_vestiaire(vestiaireco_data_airforce1_enfant)
df_nike_blazer_Enfant

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation
0,NIKE Blazer en cuir Baskets,94.50,NIKE,Vert,8,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie
1,NIKE Blazer en cuir Baskets,63.00,NIKE,Rose,3.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie
2,NIKE Blazer Baskets,42.00,NIKE,Gris,38,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Italie
3,NIKE Blazer en cuir verni Baskets,52.50,NIKE,Blanc,36,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Pays-Bas
4,NIKE Baskets Blazer en cuir,68.25,NIKE,Blanc,25,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Italie
5,NIKE Baskets Blazer en cuir,47.25,NIKE,Multicolore,36,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Italie
6,NIKE Baskets Blazer en cuir,79.80,NIKE,Blanc,36,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Pays-Bas
7,NIKE Baskets Blazer en cuir,48.30,NIKE,Violet,38,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Italie
8,NIKE Baskets Blazer en cuir,42.00,NIKE,Blanc,4.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Italie
9,NIKE Baskets Blazer en cuir,93.56,NIKE,Blanc,1,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Etats-Unis


In [31]:
df_nike_blazer_Enfant['Gender']="Kid"

In [32]:
df_nike_blazer_Enfant.to_csv("vc_data_NikeBlazer_ENFANT.csv",index=False)

##### Concataned Dataframe of Nike Blazer

In [33]:
df_nike_blazer_Femme = pd.read_csv("vc_data_NikeBlazer_FEMME.csv",sep=",")
df_nike_blazer_Homme = pd.read_csv("vc_data_NikeBlazer_HOMME.csv",sep=",")
df_nike_blazer_Enfant = pd.read_csv("vc_data_NikeBlazer_ENFANT.csv",sep=",")

In [34]:
df_nike_blazer = pd.concat([df_nike_blazer_Femme, df_nike_blazer_Homme, df_nike_blazer_Enfant], axis=0, ignore_index=True)
df_nike_blazer

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender
0,NIKE Blazer Baskets,63.00,NIKE,Blanc,38.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie,Women
1,NIKE Blazer Baskets,42.00,NIKE,Bordeaux,6.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Irlande,Women
2,NIKE Blazer Baskets,63.00,NIKE,Blanc,41.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie,Women
3,NIKE Blazer Baskets,55.65,NIKE,Vert,39.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie,Women
4,NIKE Blazer en toile Baskets,42.00,NIKE,Noir,38.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie,Women
...,...,...,...,...,...,...,...,...,...
549,NIKE Baskets Blazer en cuir,93.56,NIKE,Blanc,1.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Etats-Unis,Kid
550,NIKE En cuir Baskets,42.00,NIKE,Blanc,5.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie,Kid
551,NIKE En cuir vegan Baskets,63.00,NIKE,Rose,35.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie,Kid
552,NIKE Baskets en cuir,47.25,NIKE,Noir,4.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Italie,Kid


In [35]:
# Adding columns to the DataFrame for the approximate carbon footprint and Nike's fixed value
df_nike_blazer["Approximate Carbon Footprint (kg CO2)"] = df_nike_blazer["Localisation"].apply(calculate_approximate_carbon_footprint)
df_nike_blazer["Approximative Carbon Print from Ordering on Nike (kg CO2/ton)"] = 405  

# Adding a column for Carbon Profit
df_nike_blazer["Carbon Profit (kg CO2)"] = df_nike_blazer["Approximative Carbon Print from Ordering on Nike (kg CO2/ton)"] - df_nike_blazer["Approximate Carbon Footprint (kg CO2)"]

df_nike_blazer

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender,Approximate Carbon Footprint (kg CO2),Approximative Carbon Print from Ordering on Nike (kg CO2/ton),Carbon Profit (kg CO2)
0,NIKE Blazer Baskets,63.00,NIKE,Blanc,38.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie,Women,211.591248,405,193.408752
1,NIKE Blazer Baskets,42.00,NIKE,Bordeaux,6.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Irlande,Women,170.405381,405,234.594619
2,NIKE Blazer Baskets,63.00,NIKE,Blanc,41.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie,Women,211.591248,405,193.408752
3,NIKE Blazer Baskets,55.65,NIKE,Vert,39.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie,Women,211.591248,405,193.408752
4,NIKE Blazer en toile Baskets,42.00,NIKE,Noir,38.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie,Women,211.591248,405,193.408752
...,...,...,...,...,...,...,...,...,...,...,...,...
549,NIKE Baskets Blazer en cuir,93.56,NIKE,Blanc,1.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Etats-Unis,Kid,1522.772797,405,-1117.772797
550,NIKE En cuir Baskets,42.00,NIKE,Blanc,5.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie,Kid,211.591248,405,193.408752
551,NIKE En cuir vegan Baskets,63.00,NIKE,Rose,35.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie,Kid,211.591248,405,193.408752
552,NIKE Baskets en cuir,47.25,NIKE,Noir,4.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Italie,Kid,211.591248,405,193.408752


In [38]:
url2 = "https://www.nike.com/fr/w?q=nike%20blazer&vst=nike%20blazer"
average_nike_price2=get_nike_prices(url2)
average_nike_price2

92.13399999999993

In [39]:
df_nike_blazer.sort_values(by='Price', inplace=True)

# Calculate profit made for Vinted data
df_nike_blazer['Average price of the product on Nike'] = average_nike_price2
df_nike_blazer['Profit Made'] = df_nike_blazer.apply(lambda row: calculate_profit_made(row, average_nike_price2), axis=1)

df_nike_blazer

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender,Approximate Carbon Footprint (kg CO2),Approximative Carbon Print from Ordering on Nike (kg CO2/ton),Carbon Profit (kg CO2),Average price of the product on Nike,Profit Made
77,NIKE Baskets Blazer,15.75,NIKE,Bordeaux,38.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Italie,Women,211.591248,405,193.408752,92.134,76.384
43,NIKE Blazer en velours Baskets,15.75,NIKE,Noir,38.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,France,Women,50.586158,405,354.413842,92.134,76.384
278,NIKE Baskets en toile,16.80,NIKE,Violet,38.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Grèce,Women,382.200731,405,22.799269,92.134,75.334
59,NIKE Baskets Blazer,18.90,NIKE,Beige,36.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Italie,Women,211.591248,405,193.408752,92.134,73.234
380,NIKE Baskets montantes Blazer,19.95,NIKE,Noir,7.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Irlande,Men,170.405381,405,234.594619,92.134,72.184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525,NIKE Baskets montantes en cuir,630.00,NIKE,Noir,10.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Italie,Men,211.591248,405,193.408752,92.134,-537.866
518,NIKE Baskets montantes en cuir,630.00,NIKE,Noir,11.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Italie,Men,211.591248,405,193.408752,92.134,-537.866
473,NIKE Blazer Baskets,945.02,NIKE,Bleu,7.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Etats-Unis,Men,1522.772797,405,-1117.772797,92.134,-852.886
475,NIKE Blazer en cuir,1069.77,NIKE,Blanc,5.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Royaume-Uni,Men,151.407889,405,253.592111,92.134,-977.636


In [40]:
df_nike_blazer= df_nike_blazer.dropna(subset=['Profit Made'])
df_nike_blazer

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender,Approximate Carbon Footprint (kg CO2),Approximative Carbon Print from Ordering on Nike (kg CO2/ton),Carbon Profit (kg CO2),Average price of the product on Nike,Profit Made
77,NIKE Baskets Blazer,15.75,NIKE,Bordeaux,38.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Italie,Women,211.591248,405,193.408752,92.134,76.384
43,NIKE Blazer en velours Baskets,15.75,NIKE,Noir,38.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,France,Women,50.586158,405,354.413842,92.134,76.384
278,NIKE Baskets en toile,16.80,NIKE,Violet,38.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Grèce,Women,382.200731,405,22.799269,92.134,75.334
59,NIKE Baskets Blazer,18.90,NIKE,Beige,36.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Italie,Women,211.591248,405,193.408752,92.134,73.234
380,NIKE Baskets montantes Blazer,19.95,NIKE,Noir,7.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Irlande,Men,170.405381,405,234.594619,92.134,72.184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525,NIKE Baskets montantes en cuir,630.00,NIKE,Noir,10.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Italie,Men,211.591248,405,193.408752,92.134,-537.866
518,NIKE Baskets montantes en cuir,630.00,NIKE,Noir,11.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Italie,Men,211.591248,405,193.408752,92.134,-537.866
473,NIKE Blazer Baskets,945.02,NIKE,Bleu,7.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Etats-Unis,Men,1522.772797,405,-1117.772797,92.134,-852.886
475,NIKE Blazer en cuir,1069.77,NIKE,Blanc,5.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Royaume-Uni,Men,151.407889,405,253.592111,92.134,-977.636


In [41]:
df_nike_blazer.to_csv("VC_AllData_NikeBlazer.csv",index=False)

### Nike Dunk

##### Femme

In [42]:
url = "https://fr.vestiairecollective.com/search/?q=nike+dunk#gender=Femme%231_categoryParent=Chaussures%233_condition=Jamais%20port%C3%A9%20avec%20%C3%A9tiquette%231-Jamais%20port%C3%A9%232-Tr%C3%A8s%20bon%20%C3%A9tat%233"
vestiaireco_data_dunk_femme = get_vestiaire_data(url,2000)
df_nike_dunk_Femme = process_data_vestiaire(vestiaireco_data_dunk_femme)
df_nike_dunk_Femme

Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping con

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation
0,NIKE SB Dunk Baskets,94.50,NIKE,Rouge,6.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie
1,NIKE SB Dunk Baskets,115.50,NIKE,Blanc,40.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,France
2,NIKE SB Dunk Baskets,73.50,NIKE,Rose,42,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie
3,NIKE Dunk Sky Baskets,36.75,NIKE,Autre,38.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie
4,NIKE SB Dunk Baskets,61.95,NIKE,Kaki,40,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie
...,...,...,...,...,...,...,...,...
867,NIKE Baskets SB Dunk Low en cuir,189.00,NIKE,Bleu,38.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Espagne
868,NIKE Baskets SB Dunk Low en cuir,220.50,NIKE,Camel,41,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Italie
869,NIKE Baskets SB Dunk Low en cuir,252.00,NIKE,Bordeaux,40.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,France
870,NIKE Baskets SB Dunk Low en cuir,240.45,NIKE,Multicolore,38.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Pays-Bas


In [43]:
df_nike_dunk_Femme['Gender']="Women"

In [44]:
df_nike_dunk_Femme.to_csv("vc_data_NikeDunk_FEMME.csv",index=False)

##### Homme

In [45]:
url = "https://fr.vestiairecollective.com/search/?q=nike+dunk#condition=Jamais%20port%C3%A9%20avec%20%C3%A9tiquette%231-Jamais%20port%C3%A9%232-Tr%C3%A8s%20bon%20%C3%A9tat%233_gender=Homme%232_categoryParent=Chaussures%2313_brand=Nike%2361"
vestiaireco_data_dunk_homme = get_vestiaire_data(url,1900)
df_nike_dunk_Homme = process_data_vestiaire(vestiaireco_data_dunk_homme)
df_nike_dunk_Homme

Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping con

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation
0,NIKE SB Dunk,78.75,NIKE,Autre,42,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie
1,NIKE SB Dunk,157.50,NIKE,Blanc,44,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie
2,NIKE SB Dunk,357.00,NIKE,Rose,44,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie
3,NIKE SB Dunk,103.95,NIKE,Bleu,41,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,France
4,NIKE SB Dunk,69.30,NIKE,Bleu,40.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Italie
...,...,...,...,...,...,...,...,...
958,NIKE SB Dunk Low en cuir,577.50,NIKE,Rose,42.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Espagne
959,NIKE Baskets basses SB Dunk Low,619.50,NIKE,Bleu,45.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,France
960,NIKE Baskets basses SB Dunk Low,724.50,NIKE,Multicolore,41,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,France
961,NIKE Baskets basses SB Dunk Low en cuir,222.60,NIKE,Marron,43,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Italie


In [46]:
df_nike_dunk_Homme['Gender']="Men"

In [47]:
df_nike_dunk_Homme.to_csv("vc_data_NikeDunk_HOMME.csv",index=False)

##### Enfant

In [49]:
url = "https://fr.vestiairecollective.com/search/?q=nike+dunk#condition=Jamais%20port%C3%A9%20avec%20%C3%A9tiquette%231-Jamais%20port%C3%A9%232-Tr%C3%A8s%20bon%20%C3%A9tat%233_brand=Nike%2361_gender=Enfant%233_categoryParent=Chaussures%2311"
vestiaireco_data_dunk_enfant = get_vestiaire_data(url,129)
df_nike_dunk_Enfant = process_data_vestiaire(vestiaireco_data_dunk_enfant)
df_nike_dunk_Enfant

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation
0,NIKE SB Dunk Baskets,261.45,NIKE,Rose,18,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie
1,NIKE SB Dunk Baskets,84.00,NIKE,Bleu,37,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,France
2,NIKE SB Dunk en cuir Baskets,52.50,NIKE,Noir,7.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie
3,NIKE SB Dunk en cuir Baskets,68.25,NIKE,Noir,37,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Monaco
4,NIKE SB Dunk en cuir Baskets,47.25,NIKE,Blanc,28,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Italie
...,...,...,...,...,...,...,...,...
121,NIKE Baskets en cuir,40.12,NIKE,Multicolore,5.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Royaume-Uni
122,NIKE Baskets,83.72,NIKE,Bleu,1,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Royaume-Uni
123,NIKE Baskets en velours,120.35,NIKE,Marron,13.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Royaume-Uni
124,NIKE Baskets en cuir,315.36,NIKE,Multicolore,7,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Etats-Unis


In [50]:
df_nike_dunk_Enfant['Gender']="Kid"

In [51]:
df_nike_dunk_Enfant.to_csv("vc_data_NikeDunk_ENFANT.csv",index=False)

##### Concataned Dataframe of Nike Blazer

In [52]:
df_nike_dunk_Femme = pd.read_csv("vc_data_NikeDunk_FEMME.csv",sep=",")
df_nike_dunk_Homme = pd.read_csv("vc_data_NikeDunk_HOMME.csv",sep=",")
df_nike_dunk_Enfant = pd.read_csv("vc_data_NikeDunk_ENFANT.csv",sep=",")

In [53]:
df_nike_dunk = pd.concat([df_nike_dunk_Femme, df_nike_dunk_Homme, df_nike_dunk_Enfant], axis=0, ignore_index=True)
df_nike_dunk

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender
0,NIKE SB Dunk Baskets,94.50,NIKE,Rouge,6.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie,Women
1,NIKE SB Dunk Baskets,115.50,NIKE,Blanc,40.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,France,Women
2,NIKE SB Dunk Baskets,73.50,NIKE,Rose,42.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie,Women
3,NIKE Dunk Sky Baskets,36.75,NIKE,Autre,38.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie,Women
4,NIKE SB Dunk Baskets,61.95,NIKE,Kaki,40.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie,Women
...,...,...,...,...,...,...,...,...,...
1956,NIKE Baskets en cuir,40.12,NIKE,Multicolore,5.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Royaume-Uni,Kid
1957,NIKE Baskets,83.72,NIKE,Bleu,1.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Royaume-Uni,Kid
1958,NIKE Baskets en velours,120.35,NIKE,Marron,13.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Royaume-Uni,Kid
1959,NIKE Baskets en cuir,315.36,NIKE,Multicolore,7.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Etats-Unis,Kid


In [55]:
# Adding columns to the DataFrame for the approximate carbon footprint and Nike's fixed value
df_nike_dunk["Approximate Carbon Footprint (kg CO2)"] = df_nike_dunk["Localisation"].apply(calculate_approximate_carbon_footprint)
df_nike_dunk["Approximative Carbon Print from Ordering on Nike (kg CO2/ton)"] = 405  

# Adding a column for Carbon Profit
df_nike_dunk["Carbon Profit (kg CO2)"] = df_nike_dunk["Approximative Carbon Print from Ordering on Nike (kg CO2/ton)"] - df_nike_dunk["Approximate Carbon Footprint (kg CO2)"]

df_nike_dunk


,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender,Approximate Carbon Footprint (kg CO2),Approximative Carbon Print from Ordering on Nike (kg CO2/ton),Carbon Profit (kg CO2)
0,NIKE SB Dunk Baskets,94.50,NIKE,Rouge,6.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie,Women,211.591248,405,193.408752
1,NIKE SB Dunk Baskets,115.50,NIKE,Blanc,40.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,France,Women,50.586158,405,354.413842
2,NIKE SB Dunk Baskets,73.50,NIKE,Rose,42.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie,Women,211.591248,405,193.408752
3,NIKE Dunk Sky Baskets,36.75,NIKE,Autre,38.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie,Women,211.591248,405,193.408752
4,NIKE SB Dunk Baskets,61.95,NIKE,Kaki,40.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie,Women,211.591248,405,193.408752
...,...,...,...,...,...,...,...,...,...,...,...,...
1956,NIKE Baskets en cuir,40.12,NIKE,Multicolore,5.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Royaume-Uni,Kid,151.407889,405,253.592111
1957,NIKE Baskets,83.72,NIKE,Bleu,1.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Royaume-Uni,Kid,151.407889,405,253.592111
1958,NIKE Baskets en velours,120.35,NIKE,Marron,13.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Royaume-Uni,Kid,151.407889,405,253.592111
1959,NIKE Baskets en cuir,315.36,NIKE,Multicolore,7.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Etats-Unis,Kid,1522.772797,405,-1117.772797


In [56]:
url3 = "https://www.nike.com/fr/w?q=nike%20dunk&vst=nike%20dunk"
average_nike_price3=get_nike_prices(url3)
average_nike_price3

115.2435483870967

In [57]:
df_nike_dunk.sort_values(by='Price', inplace=True)

# Calculate profit made for Vinted data
df_nike_dunk['Average price of the product on Nike'] = average_nike_price3
df_nike_dunk['Profit Made'] = df_nike_dunk.apply(lambda row: calculate_profit_made(row, average_nike_price3), axis=1)

df_nike_dunk

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender,Approximate Carbon Footprint (kg CO2),Approximative Carbon Print from Ordering on Nike (kg CO2/ton),Carbon Profit (kg CO2),Average price of the product on Nike,Profit Made
505,NIKE SB Dunk Low en cuir Baskets,18.92,NIKE,Gris,37.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,France,Women,50.586158,405,354.413842,115.243548,96.323548
1923,NIKE Baskets,21.00,NIKE,Bleu,11.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie,Kid,211.591248,405,193.408752,115.243548,94.243548
885,NIKE SB Dunk,21.00,NIKE,Bleu,43.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Italie,Men,211.591248,405,193.408752,115.243548,94.243548
639,NIKE SB Dunk Low en cuir Baskets,26.25,NIKE,Jaune,38.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Italie,Women,211.591248,405,193.408752,115.243548,88.993548
1871,NIKE Baskets SB Dunk en cuir,30.86,NIKE,Multicolore,11.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Pologne,Kid,1048.754100,405,-643.754100,115.243548,84.383548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821,NIKE SB Dunk Low en cuir,2677.50,NIKE,Bleu,43.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie,Men,211.591248,405,193.408752,115.243548,-2562.256452
1798,NIKE SB Dunk Low en cuir,4095.00,NIKE,Rouge,43.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie,Men,211.591248,405,193.408752,115.243548,-3979.756452
1764,NIKE SB Dunk Low en cuir,4252.50,NIKE,Marron,44.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie,Men,211.591248,405,193.408752,115.243548,-4137.256452
1408,NIKE SB Dunk Low en cuir,5250.00,NIKE,Marron,40.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,France,Men,50.586158,405,354.413842,115.243548,-5134.756452


In [58]:
df_nike_dunk= df_nike_dunk.dropna(subset=['Profit Made'])
df_nike_dunk

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender,Approximate Carbon Footprint (kg CO2),Approximative Carbon Print from Ordering on Nike (kg CO2/ton),Carbon Profit (kg CO2),Average price of the product on Nike,Profit Made
505,NIKE SB Dunk Low en cuir Baskets,18.92,NIKE,Gris,37.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,France,Women,50.586158,405,354.413842,115.243548,96.323548
1923,NIKE Baskets,21.00,NIKE,Bleu,11.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie,Kid,211.591248,405,193.408752,115.243548,94.243548
885,NIKE SB Dunk,21.00,NIKE,Bleu,43.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Italie,Men,211.591248,405,193.408752,115.243548,94.243548
639,NIKE SB Dunk Low en cuir Baskets,26.25,NIKE,Jaune,38.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Italie,Women,211.591248,405,193.408752,115.243548,88.993548
1871,NIKE Baskets SB Dunk en cuir,30.86,NIKE,Multicolore,11.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Pologne,Kid,1048.754100,405,-643.754100,115.243548,84.383548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821,NIKE SB Dunk Low en cuir,2677.50,NIKE,Bleu,43.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie,Men,211.591248,405,193.408752,115.243548,-2562.256452
1798,NIKE SB Dunk Low en cuir,4095.00,NIKE,Rouge,43.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie,Men,211.591248,405,193.408752,115.243548,-3979.756452
1764,NIKE SB Dunk Low en cuir,4252.50,NIKE,Marron,44.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie,Men,211.591248,405,193.408752,115.243548,-4137.256452
1408,NIKE SB Dunk Low en cuir,5250.00,NIKE,Marron,40.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,France,Men,50.586158,405,354.413842,115.243548,-5134.756452


In [59]:
df_nike_dunk.to_csv("VC_AllData_NikeDunk.csv",index=False)

### Nike Mag

##### Femme

In [60]:
url = "https://fr.vestiairecollective.com/search/?q=nike+mag#gender=Femme%231_categoryParent=Chaussures%233"
vestiaireco_data_mag_femme = get_vestiaire_data(url,6)
df_nike_mag_Femme = process_data_vestiaire(vestiaireco_data_mag_femme)
df_nike_mag_Femme

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation
0,NIKE En cuir Baskets,118.78,NIKE,Gris,9.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Etats-Unis
1,JORDAN En cuir Baskets,141.91,JORDAN,Noir,8,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Etats-Unis
2,JORDAN En cuir Baskets,141.91,JORDAN,Rouge,9,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Etats-Unis
3,JORDAN En cuir Baskets,146.12,JORDAN,Noir,8.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Etats-Unis
4,JORDAN En cuir Baskets,146.12,JORDAN,Noir,9.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Etats-Unis
5,JORDAN En cuir Baskets,156.63,JORDAN,Noir,9,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Etats-Unis


In [61]:
df_nike_mag_Femme['Gender']="Women"

In [62]:
df_nike_mag_Femme.to_csv("vc_data_NikeMag_FEMME.csv",index=False)

##### Homme

In [63]:
url = "https://fr.vestiairecollective.com/search/?q=nike+mag#gender=Homme%232_brand=Nike%2361_categoryParent=Chaussures%2313"
vestiaireco_data_mag_homme = get_vestiaire_data(url,1)
df_nike_mag_Homme = process_data_vestiaire(vestiaireco_data_mag_homme)
df_nike_mag_Homme

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation
0,NIKE Baskets montantes en toile,1048.95,NIKE,Gris,42,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,France


In [64]:
df_nike_mag_Homme['Gender']="Men"

In [65]:
df_nike_mag_Homme.to_csv("vc_data_NikeMag_HOMME.csv",index=False)

##### Concataned Dataframe of Nike Mag

In [66]:
df_nike_mag_Femme = pd.read_csv("vc_data_NikeMag_FEMME.csv",sep=",")
df_nike_mag_Homme = pd.read_csv("vc_data_NikeMag_HOMME.csv",sep=",")

In [67]:
df_nike_mag = pd.concat([df_nike_mag_Femme, df_nike_mag_Homme], axis=0, ignore_index=True)
df_nike_mag

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender
0,NIKE En cuir Baskets,118.78,NIKE,Gris,9.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Etats-Unis,Women
1,JORDAN En cuir Baskets,141.91,JORDAN,Noir,8.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Etats-Unis,Women
2,JORDAN En cuir Baskets,141.91,JORDAN,Rouge,9.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Etats-Unis,Women
3,JORDAN En cuir Baskets,146.12,JORDAN,Noir,8.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Etats-Unis,Women
4,JORDAN En cuir Baskets,146.12,JORDAN,Noir,9.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Etats-Unis,Women
5,JORDAN En cuir Baskets,156.63,JORDAN,Noir,9.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Etats-Unis,Women
6,NIKE Baskets montantes en toile,1048.95,NIKE,Gris,42.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,France,Men


In [68]:
# Adding columns to the DataFrame for the approximate carbon footprint and Nike's fixed value
df_nike_mag["Approximate Carbon Footprint (kg CO2)"] = df_nike_mag["Localisation"].apply(calculate_approximate_carbon_footprint)
df_nike_mag["Approximative Carbon Print from Ordering on Nike (kg CO2/ton)"] = 405  

# Adding a column for Carbon Profit
df_nike_mag["Carbon Profit (kg CO2)"] = df_nike_mag["Approximative Carbon Print from Ordering on Nike (kg CO2/ton)"] - df_nike_mag["Approximate Carbon Footprint (kg CO2)"]

df_nike_mag

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender,Approximate Carbon Footprint (kg CO2),Approximative Carbon Print from Ordering on Nike (kg CO2/ton),Carbon Profit (kg CO2)
0,NIKE En cuir Baskets,118.78,NIKE,Gris,9.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Etats-Unis,Women,1522.772797,405,-1117.772797
1,JORDAN En cuir Baskets,141.91,JORDAN,Noir,8.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Etats-Unis,Women,1522.772797,405,-1117.772797
2,JORDAN En cuir Baskets,141.91,JORDAN,Rouge,9.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Etats-Unis,Women,1522.772797,405,-1117.772797
3,JORDAN En cuir Baskets,146.12,JORDAN,Noir,8.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Etats-Unis,Women,1522.772797,405,-1117.772797
4,JORDAN En cuir Baskets,146.12,JORDAN,Noir,9.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Etats-Unis,Women,1522.772797,405,-1117.772797
5,JORDAN En cuir Baskets,156.63,JORDAN,Noir,9.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Etats-Unis,Women,1522.772797,405,-1117.772797
6,NIKE Baskets montantes en toile,1048.95,NIKE,Gris,42.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,France,Men,50.586158,405,354.413842


In [69]:
url4 = "https://www.nike.com/fr/w?q=nike%20mag&vst=nike%20mag"
average_nike_price4=get_nike_prices(url4)
average_nike_price4

111.48555555555556

In [70]:
df_nike_mag.sort_values(by='Price', inplace=True)

# Calculate profit made for Vinted data
df_nike_mag['Average price of the product on Nike'] = average_nike_price4
df_nike_mag['Profit Made'] = df_nike_mag.apply(lambda row: calculate_profit_made(row, average_nike_price4), axis=1)

df_nike_mag

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender,Approximate Carbon Footprint (kg CO2),Approximative Carbon Print from Ordering on Nike (kg CO2/ton),Carbon Profit (kg CO2),Average price of the product on Nike,Profit Made
0,NIKE En cuir Baskets,118.78,NIKE,Gris,9.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Etats-Unis,Women,1522.772797,405,-1117.772797,111.485556,-7.294444
1,JORDAN En cuir Baskets,141.91,JORDAN,Noir,8.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Etats-Unis,Women,1522.772797,405,-1117.772797,111.485556,-30.424444
2,JORDAN En cuir Baskets,141.91,JORDAN,Rouge,9.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Etats-Unis,Women,1522.772797,405,-1117.772797,111.485556,-30.424444
3,JORDAN En cuir Baskets,146.12,JORDAN,Noir,8.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Etats-Unis,Women,1522.772797,405,-1117.772797,111.485556,-34.634444
4,JORDAN En cuir Baskets,146.12,JORDAN,Noir,9.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Etats-Unis,Women,1522.772797,405,-1117.772797,111.485556,-34.634444
5,JORDAN En cuir Baskets,156.63,JORDAN,Noir,9.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Etats-Unis,Women,1522.772797,405,-1117.772797,111.485556,-45.144444
6,NIKE Baskets montantes en toile,1048.95,NIKE,Gris,42.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,France,Men,50.586158,405,354.413842,111.485556,-937.464444


In [71]:
df_nike_mag= df_nike_mag.dropna(subset=['Profit Made'])
df_nike_mag

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender,Approximate Carbon Footprint (kg CO2),Approximative Carbon Print from Ordering on Nike (kg CO2/ton),Carbon Profit (kg CO2),Average price of the product on Nike,Profit Made
0,NIKE En cuir Baskets,118.78,NIKE,Gris,9.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Etats-Unis,Women,1522.772797,405,-1117.772797,111.485556,-7.294444
1,JORDAN En cuir Baskets,141.91,JORDAN,Noir,8.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Etats-Unis,Women,1522.772797,405,-1117.772797,111.485556,-30.424444
2,JORDAN En cuir Baskets,141.91,JORDAN,Rouge,9.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Etats-Unis,Women,1522.772797,405,-1117.772797,111.485556,-30.424444
3,JORDAN En cuir Baskets,146.12,JORDAN,Noir,8.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Etats-Unis,Women,1522.772797,405,-1117.772797,111.485556,-34.634444
4,JORDAN En cuir Baskets,146.12,JORDAN,Noir,9.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Etats-Unis,Women,1522.772797,405,-1117.772797,111.485556,-34.634444
5,JORDAN En cuir Baskets,156.63,JORDAN,Noir,9.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Etats-Unis,Women,1522.772797,405,-1117.772797,111.485556,-45.144444
6,NIKE Baskets montantes en toile,1048.95,NIKE,Gris,42.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,France,Men,50.586158,405,354.413842,111.485556,-937.464444


In [72]:
df_nike_mag.to_csv("VC_AllData_NikeMag.csv",index=False)

### Nike Air Max 1 

##### Femme

In [73]:
url = "https://fr.vestiairecollective.com/search/?q=nike+air+max+1#gender=Femme%231_categoryParent=Chaussures%233_brand=Nike%2361_condition=Jamais%20port%C3%A9%20avec%20%C3%A9tiquette%231-Jamais%20port%C3%A9%232-Tr%C3%A8s%20bon%20%C3%A9tat%233"
vestiaireco_data_airmax1_femme = get_vestiaire_data(url,2800)
df_nike_airmax1_Femme = process_data_vestiaire(vestiaireco_data_airmax1_femme)
df_nike_airmax1_Femme

Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping con

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation
0,NIKE Air Max 1 Baskets,170.10,NIKE,Multicolore,40,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Espagne
1,NIKE Air Max 1 Baskets,81.90,NIKE,Argenté,38.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie
2,NIKE Air Max 1 Baskets,37.80,NIKE,Noir,38,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Allemagne
3,NIKE Air Max 1 Baskets,210.00,NIKE,Rouge,40.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,France
4,NIKE Air Max 1 Baskets,26.25,NIKE,Noir,39,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Allemagne
...,...,...,...,...,...,...,...,...
814,NIKE Air Max 97 Baskets,66.15,NIKE,Multicolore,38.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Espagne
815,NIKE Baskets Air Max 90 en cuir,59.85,NIKE,Argenté,38,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,France
816,NIKE Baskets Air Force 1 en cuir,241.50,NIKE,Beige,40,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Lituanie
817,NIKE Baskets Air Max 90 en toile,78.75,NIKE,Bleu,39,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,France


In [74]:
df_nike_airmax1_Femme['Gender']="Women"

In [75]:
df_nike_airmax1_Femme.to_csv("vc_data_NikeAirmax1_FEMME.csv",index=False)

##### Homme

In [76]:
url = "https://fr.vestiairecollective.com/search/?q=nike+air+max+1#gender=Homme%232_categoryParent=Chaussures%2313_category=13%20%3E%20Baskets%2330_brand=Nike%2361_condition=Jamais%20port%C3%A9%20avec%20%C3%A9tiquette%231-Jamais%20port%C3%A9%232-Tr%C3%A8s%20bon%20%C3%A9tat%233"
vestiaireco_data_airmax1_homme = get_vestiaire_data(url,2400)
df_nike_airmax1_Homme = process_data_vestiaire(vestiaireco_data_airmax1_homme)
df_nike_airmax1_Homme

Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping container.
Stale Element Reference Exception - Skipping con

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation
0,NIKE Air Max 1,134.40,NIKE,Bleu,40,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Espagne
1,NIKE Air Max 1,371.70,NIKE,Multicolore,45.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Espagne
2,NIKE Air Max 1,229.95,NIKE,Noir,42.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Roumanie
3,NIKE Air Max 1,210.00,NIKE,Écru,9,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Allemagne
4,NIKE Air Max 1,123.90,NIKE,Blanc,44,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,France
...,...,...,...,...,...,...,...,...
909,NIKE Baskets montantes Air Force 1 en cuir,136.50,NIKE,Bordeaux,41,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Espagne
910,NIKE Baskets basses Air Force 1 en cuir,115.50,NIKE,Beige,42,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Autriche
911,NIKE Baskets Air Max 2 Light,89.25,NIKE,Multicolore,8.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Italie
912,NIKE Baskets basses Air Force 1 en cuir,94.50,NIKE,Noir,42.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Italie


In [77]:
df_nike_airmax1_Homme['Gender']="Men"

In [78]:
df_nike_airmax1_Homme.to_csv("vc_data_NikeAirmax1_HOMME.csv",index=False)

##### Enfant

In [79]:
url = "https://fr.vestiairecollective.com/search/?q=nike+air+max+1#gender=Enfant%233_brand=Nike%2361_condition=Jamais%20port%C3%A9%20avec%20%C3%A9tiquette%231-Jamais%20port%C3%A9%232-Tr%C3%A8s%20bon%20%C3%A9tat%233_categoryParent=Chaussures%2311"
vestiaireco_data_airmax1_enfant = get_vestiaire_data(url,100)
df_nike_airmax1_Enfant = process_data_vestiaire(vestiaireco_data_airmax1_enfant)
df_nike_airmax1_Enfant

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation
0,NIKE Air Max 1 Baskets,31.50,NIKE,Blanc,25,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,France
1,NIKE Air Max 1 en toile Baskets,26.28,NIKE,Multicolore,27,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Pays-Bas
2,NIKE Air Max 1 Baskets,22.05,NIKE,Blanc,26,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie
3,NIKE Air Max 1 en cuir Baskets,42.00,NIKE,Blanc,33,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Grèce
4,NIKE Baskets Air Max 1 en toile,23.10,NIKE,Multicolore,10.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Italie
...,...,...,...,...,...,...,...,...
95,NIKE Baskets Air Max 1/97 Sean Wotherspoon en ...,42.00,NIKE,Noir,38,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Hongrie
96,NIKE Air Max 97 en cuir Baskets,75.90,NIKE,Argenté,6,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Pologne
97,NIKE Baskets Air Force 1 en cuir,99.75,NIKE,Noir,36,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Pays-Bas
98,NIKE Baskets Air Force 1 en cuir,61.95,NIKE,Blanc,8.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Italie


In [80]:
df_nike_airmax1_Enfant['Gender']="Kid"

In [81]:
df_nike_airmax1_Enfant.to_csv("vc_data_NikeAirmax1_ENFANT.csv",index=False)

##### Concataned Dataframe of Nike Air Max 1

In [82]:
df_nike_airmax1_Femme = pd.read_csv("vc_data_NikeAirmax1_FEMME.csv",sep=",")
df_nike_airmax1_Homme = pd.read_csv("vc_data_NikeAirmax1_HOMME.csv",sep=",")
df_nike_airmax1_Enfant = pd.read_csv("vc_data_NikeAirmax1_ENFANT.csv",sep=",")

In [83]:
df_nike_airmax1 = pd.concat([df_nike_airmax1_Femme, df_nike_airmax1_Homme, df_nike_airmax1_Enfant], axis=0, ignore_index=True)
df_nike_airmax1

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender
0,NIKE Air Max 1 Baskets,170.10,NIKE,Multicolore,40.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Espagne,Women
1,NIKE Air Max 1 Baskets,81.90,NIKE,Argenté,38.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie,Women
2,NIKE Air Max 1 Baskets,37.80,NIKE,Noir,38.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Allemagne,Women
3,NIKE Air Max 1 Baskets,210.00,NIKE,Rouge,40.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,France,Women
4,NIKE Air Max 1 Baskets,26.25,NIKE,Noir,39.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Allemagne,Women
...,...,...,...,...,...,...,...,...,...
1828,NIKE Baskets Air Max 1/97 Sean Wotherspoon en ...,42.00,NIKE,Noir,38.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Hongrie,Kid
1829,NIKE Air Max 97 en cuir Baskets,75.90,NIKE,Argenté,6.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Pologne,Kid
1830,NIKE Baskets Air Force 1 en cuir,99.75,NIKE,Noir,36.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Pays-Bas,Kid
1831,NIKE Baskets Air Force 1 en cuir,61.95,NIKE,Blanc,8.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Italie,Kid


In [84]:
# Adding columns to the DataFrame for the approximate carbon footprint and Nike's fixed value
df_nike_airmax1["Approximate Carbon Footprint (kg CO2)"] = df_nike_airmax1["Localisation"].apply(calculate_approximate_carbon_footprint)
df_nike_airmax1["Approximative Carbon Print from Ordering on Nike (kg CO2/ton)"] = 405  

# Adding a column for Carbon Profit
df_nike_airmax1["Carbon Profit (kg CO2)"] = df_nike_airmax1["Approximative Carbon Print from Ordering on Nike (kg CO2/ton)"] - df_nike_airmax1["Approximate Carbon Footprint (kg CO2)"]

df_nike_airmax1

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender,Approximate Carbon Footprint (kg CO2),Approximative Carbon Print from Ordering on Nike (kg CO2/ton),Carbon Profit (kg CO2)
0,NIKE Air Max 1 Baskets,170.10,NIKE,Multicolore,40.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Espagne,Women,240.819687,405,164.180313
1,NIKE Air Max 1 Baskets,81.90,NIKE,Argenté,38.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie,Women,211.591248,405,193.408752
2,NIKE Air Max 1 Baskets,37.80,NIKE,Noir,38.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Allemagne,Women,40.433839,405,364.566161
3,NIKE Air Max 1 Baskets,210.00,NIKE,Rouge,40.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,France,Women,50.586158,405,354.413842
4,NIKE Air Max 1 Baskets,26.25,NIKE,Noir,39.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Allemagne,Women,40.433839,405,364.566161
...,...,...,...,...,...,...,...,...,...,...,...,...
1828,NIKE Baskets Air Max 1/97 Sean Wotherspoon en ...,42.00,NIKE,Noir,38.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Hongrie,Kid,119.482326,405,285.517674
1829,NIKE Air Max 97 en cuir Baskets,75.90,NIKE,Argenté,6.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Pologne,Kid,1048.754100,405,-643.754100
1830,NIKE Baskets Air Force 1 en cuir,99.75,NIKE,Noir,36.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Pays-Bas,Kid,87.928759,405,317.071241
1831,NIKE Baskets Air Force 1 en cuir,61.95,NIKE,Blanc,8.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Italie,Kid,211.591248,405,193.408752


In [87]:
url5 = "https://www.nike.com/fr/w?q=nike%20air%20max%201&vst=nike%20air%20max%201"
average_nike_price5=get_nike_prices(url5)
average_nike_price5

134.99941176470585

In [89]:
df_nike_airmax1.sort_values(by='Price', inplace=True)

# Calculate profit made for Vinted data
df_nike_airmax1['Average price of the product on Nike'] = average_nike_price5
df_nike_airmax1['Profit Made'] = df_nike_airmax1.apply(lambda row: calculate_profit_made(row, average_nike_price5), axis=1)

df_nike_airmax1

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender,Approximate Carbon Footprint (kg CO2),Approximative Carbon Print from Ordering on Nike (kg CO2/ton),Carbon Profit (kg CO2),Average price of the product on Nike,Profit Made
1787,NIKE Air Force 1 en cuir Baskets,16.80,NIKE,Noir,25.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Irlande,Kid,170.405381,405,234.594619,134.999412,118.199412
1811,NIKE Baskets Air Force 1 en cuir,21.00,NIKE,Blanc,22.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,France,Kid,50.586158,405,354.413842,134.999412,113.999412
1751,NIKE Baskets Air Max 1 en toile,21.00,NIKE,Noir,13.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Grèce,Kid,382.200731,405,22.799269,134.999412,113.999412
1735,NIKE Air Max 1 Baskets,22.05,NIKE,Blanc,26.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie,Kid,211.591248,405,193.408752,134.999412,112.949412
908,NIKE Air Max 1,23.10,NIKE,Noir,38.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Italie,Men,211.591248,405,193.408752,134.999412,111.899412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1654,NIKE Baskets basses Air Max 1/97 Sean Wothersp...,1482.60,NIKE,Autre,8.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,France,Men,50.586158,405,354.413842,134.999412,-1347.600588
1408,NIKE Air Force 1,1627.50,NIKE,Gris,7.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,France,Men,50.586158,405,354.413842,134.999412,-1492.500588
1497,NIKE Air Force 1 en cuir,1653.75,NIKE,Multicolore,44.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Autriche,Men,176.990280,405,228.009720,134.999412,-1518.750588
1371,NIKE Air Force 1,2097.90,NIKE,Noir,42.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,France,Men,50.586158,405,354.413842,134.999412,-1962.900588


In [90]:
df_nike_airmax1= df_nike_airmax1.dropna(subset=['Profit Made'])
df_nike_airmax1

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender,Approximate Carbon Footprint (kg CO2),Approximative Carbon Print from Ordering on Nike (kg CO2/ton),Carbon Profit (kg CO2),Average price of the product on Nike,Profit Made
1787,NIKE Air Force 1 en cuir Baskets,16.80,NIKE,Noir,25.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Irlande,Kid,170.405381,405,234.594619,134.999412,118.199412
1811,NIKE Baskets Air Force 1 en cuir,21.00,NIKE,Blanc,22.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,France,Kid,50.586158,405,354.413842,134.999412,113.999412
1751,NIKE Baskets Air Max 1 en toile,21.00,NIKE,Noir,13.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Grèce,Kid,382.200731,405,22.799269,134.999412,113.999412
1735,NIKE Air Max 1 Baskets,22.05,NIKE,Blanc,26.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie,Kid,211.591248,405,193.408752,134.999412,112.949412
908,NIKE Air Max 1,23.10,NIKE,Noir,38.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Italie,Men,211.591248,405,193.408752,134.999412,111.899412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1654,NIKE Baskets basses Air Max 1/97 Sean Wothersp...,1482.60,NIKE,Autre,8.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,France,Men,50.586158,405,354.413842,134.999412,-1347.600588
1408,NIKE Air Force 1,1627.50,NIKE,Gris,7.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,France,Men,50.586158,405,354.413842,134.999412,-1492.500588
1497,NIKE Air Force 1 en cuir,1653.75,NIKE,Multicolore,44.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Autriche,Men,176.990280,405,228.009720,134.999412,-1518.750588
1371,NIKE Air Force 1,2097.90,NIKE,Noir,42.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,France,Men,50.586158,405,354.413842,134.999412,-1962.900588


In [91]:
df_nike_airmax1.to_csv("VC_AllData_NikeAirmax1.csv",index=False)

## 5. Final Concataned Dataframs VINTED & Vestiaire Collective

In [3]:
import pandas as pd
df_nikeairforce1_vinted = pd.read_csv("AllData_Nike_AirForce1.csv",sep=",")
df_nikeairforce1VC =  pd.read_csv("VC_AllData_Nike_AirForce1.csv",sep=",")

In [4]:
df_nikeairforce1 = pd.concat([df_nikeairforce1_vinted, df_nikeairforce1VC], axis=0, ignore_index=True)
df_nikeairforce1

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender,Approximate Carbon Footprint (kg CO2),Approximative Carbon Print from Ordering on Nike (kg CO2/ton),Carbon Profit (kg CO2),Average price of the product on Nike,Profit Made
0,Air force 1,3.50,NIKE,"NOIR, BLANC",23,https://www.vinted.fr/items/3803206036-air-for...,https://images1.vinted.net/t/03_00e3f_soGZYV6Z...,"PORTO, PORTUGAL",Kid,242.903503,405,162.096497,117.331379,113.831379
1,Chaussures Nike Air Orange,3.50,NIKE,"ORANGE, BLANC",37,https://www.vinted.fr/items/3124028095-chaussu...,https://images1.vinted.net/t/01_02588_5vc8Nbbr...,"SAINTE-MAURE-DE-TOURAINE, FRANCE",Kid,46.653032,405,358.346968,117.331379,113.831379
2,Air force bébé,5.00,NIKE,BLANC,1-3 mois / 56 cm,https://www.vinted.fr/items/3649411494-air-for...,https://images1.vinted.net/t/03_026e1_1gfWBBDD...,"NORMANVILLE, FRANCE",Kid,31.657668,405,373.342332,117.331379,112.331379
3,Nike Air Force 1,5.00,NIKE,"BLEU, JAUNE","40,5",https://www.vinted.fr/items/3639577938-nike-ai...,https://images1.vinted.net/t/02_023d5_t8rfdspW...,"BILBAO, ESPAGNE",Men,148.770908,405,256.229092,117.331379,112.331379
4,Air Force 1,5.00,NIKE,BLANC,18,https://www.vinted.fr/items/3843477416-air-for...,https://images1.vinted.net/t/02_023e4_eML69SVV...,"FIGARI, FRANCE",Kid,195.212950,405,209.787050,117.331379,112.331379
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3430,NIKE Baskets en cuir,2890.77,NIKE,Multicolore,8.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Etats-Unis,Women,1522.772797,405,-1117.772797,120.296154,-2770.473846
3431,NIKE Baskets basses Air Force 1 en alligator,3889.40,NIKE,Marron,10.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Etats-Unis,Men,1522.772797,405,-1117.772797,120.296154,-3769.103846
3432,NIKE Air Force 1 en cuir,4680.23,NIKE,Vert,8.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Royaume-Uni,Men,151.407889,405,253.592111,120.296154,-4559.933846
3433,NIKE En cuir,15750.00,NIKE,Noir,10.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Italie,Men,211.591248,405,193.408752,120.296154,-15629.703846


In [5]:
df_nikeairforce1.sort_values(by='Price', inplace=True)
df_nikeairforce1

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender,Approximate Carbon Footprint (kg CO2),Approximative Carbon Print from Ordering on Nike (kg CO2/ton),Carbon Profit (kg CO2),Average price of the product on Nike,Profit Made
0,Air force 1,3.50,NIKE,"NOIR, BLANC",23,https://www.vinted.fr/items/3803206036-air-for...,https://images1.vinted.net/t/03_00e3f_soGZYV6Z...,"PORTO, PORTUGAL",Kid,242.903503,405,162.096497,117.331379,113.831379
1,Chaussures Nike Air Orange,3.50,NIKE,"ORANGE, BLANC",37,https://www.vinted.fr/items/3124028095-chaussu...,https://images1.vinted.net/t/01_02588_5vc8Nbbr...,"SAINTE-MAURE-DE-TOURAINE, FRANCE",Kid,46.653032,405,358.346968,117.331379,113.831379
2,Air force bébé,5.00,NIKE,BLANC,1-3 mois / 56 cm,https://www.vinted.fr/items/3649411494-air-for...,https://images1.vinted.net/t/03_026e1_1gfWBBDD...,"NORMANVILLE, FRANCE",Kid,31.657668,405,373.342332,117.331379,112.331379
3,Nike Air Force 1,5.00,NIKE,"BLEU, JAUNE","40,5",https://www.vinted.fr/items/3639577938-nike-ai...,https://images1.vinted.net/t/02_023d5_t8rfdspW...,"BILBAO, ESPAGNE",Men,148.770908,405,256.229092,117.331379,112.331379
4,Air Force 1,5.00,NIKE,BLANC,18,https://www.vinted.fr/items/3843477416-air-for...,https://images1.vinted.net/t/02_023e4_eML69SVV...,"FIGARI, FRANCE",Kid,195.212950,405,209.787050,117.331379,112.331379
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3430,NIKE Baskets en cuir,2890.77,NIKE,Multicolore,8.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Etats-Unis,Women,1522.772797,405,-1117.772797,120.296154,-2770.473846
3431,NIKE Baskets basses Air Force 1 en alligator,3889.40,NIKE,Marron,10.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Etats-Unis,Men,1522.772797,405,-1117.772797,120.296154,-3769.103846
3432,NIKE Air Force 1 en cuir,4680.23,NIKE,Vert,8.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Royaume-Uni,Men,151.407889,405,253.592111,120.296154,-4559.933846
3433,NIKE En cuir,15750.00,NIKE,Noir,10.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Italie,Men,211.591248,405,193.408752,120.296154,-15629.703846


In [6]:
df_nikeairforce1.to_csv("FINAL_AllData_Nike_AirForce1.csv",index=False)

In [7]:
df_nikeblazer_vinted = pd.read_csv("AllData_NikeBlazer.csv",sep=";")
df_nikeblazerVC =  pd.read_csv("VC_AllData_NikeBlazer.csv",sep=",")

In [8]:
df_nikeblazer = pd.concat([df_nikeblazer_vinted, df_nikeblazerVC], axis=0, ignore_index=True)
df_nikeblazer

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender,Approximate Carbon Footprint (kg CO2),Approximative Carbon Print from Ordering on Nike (kg CO2/ton),Carbon Profit (kg CO2),Average price of the product on Nike,Profit Made,Unnamed: 14
0,Baskets blazer low,3.0,NIKE,BLANC,29,https://www.vinted.fr/items/3853744571-baskets...,https://images1.vinted.net/t/03_00862_ZgCjuxpW...,"RAISMES, FRANCE",Kid,37.849440217816145,405.0,367.150560,88.685676,85.685676,NaN
1,Nike blazer,4.0,NIKE,"GRIS, ROSE",37.5,https://www.vinted.fr/items/276833690-nike-bla...,https://images1.vinted.net/t/01_00ef1_Aip1r7zu...,"MARSEILLE, FRANCE",Women,132.10630160408124,405.0,272.893698,88.685676,84.685676,NaN
2,Nike blazer,5.0,NIKE,"BLEU, CRÈME",39.0,https://www.vinted.fr/items/920130087-nike-bla...,https://images1.vinted.net/t/01_002f0_Bf4V5r85...,FRANCE,Women,50.58615778249138,405.0,354.413842,88.685676,83.685676,NaN
3,Blazer nike,5.0,NIKE,"GRIS, BLANC",37.5,https://www.vinted.fr/items/3181960447-blazer-...,https://images1.vinted.net/t/02_0152d_yd3gbWfF...,FRANCE,Women,50.58615778249138,405.0,354.413842,88.685676,83.685676,NaN
4,Nike blazer,5.0,NIKE,"GRIS, ROSE",36.0,https://www.vinted.fr/items/1619529192-nike-bl...,https://images1.vinted.net/t/03_000a1_ez3c2r54...,"SANGATTE, FRANCE",Women,47.2636523312419,405.0,357.736348,88.685676,83.685676,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2628,NIKE Baskets montantes en cuir,630.0,NIKE,Noir,10.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Italie,Men,211.591248,405.0,193.408752,92.134000,-537.866000,NaN
2629,NIKE Baskets montantes en cuir,630.0,NIKE,Noir,11.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Italie,Men,211.591248,405.0,193.408752,92.134000,-537.866000,NaN
2630,NIKE Blazer Baskets,945.02,NIKE,Bleu,7.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Etats-Unis,Men,1522.772797,405.0,-1117.772797,92.134000,-852.886000,NaN
2631,NIKE Blazer en cuir,1069.77,NIKE,Blanc,5.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Royaume-Uni,Men,151.407889,405.0,253.592111,92.134000,-977.636000,NaN


In [9]:
df_nikeblazer.to_csv("FINAL_AllData_NikeBlazer.csv",index=False)

In [10]:
df_nikedunk_vinted = pd.read_csv("AllData_NikeDunk.csv",sep=",")
df_nikedunkVC =  pd.read_csv("VC_AllData_NikeDunk.csv",sep=",")

In [11]:
df_nikedunk = pd.concat([df_nikedunk_vinted, df_nikedunkVC], axis=0, ignore_index=True)
df_nikedunk

,Unnamed: 0,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender,Approximate Carbon Footprint (kg CO2),Approximative Carbon Print from Ordering on Nike (kg CO2/ton),Carbon Profit (kg CO2),Average price of the product on Nike,Profit Made
0,0.0,Dunk low,1.0,NIKE,"BLANC, BLEU","44,5",https://www.vinted.fr/items/3890663184-dunk-lo...,https://images1.vinted.net/t/03_0186f_nVyJSyue...,"GIF-SUR-YVETTE, FRANCE",Men,4.698729,405,400.301271,115.888710,114.888710
1,51.0,Stikers autocollants Nike Dunk low Off-White e...,3.5,NIKE,MULTICOLORE,Taille non spécifiée,https://www.vinted.fr/items/3832550323-stikers...,https://images1.vinted.net/t/02_003d4_6uf2Pfz6...,FRANCE,Kid,50.586158,405,354.413842,115.888710,112.388710
2,49.0,dunk low,5.0,NIKE,"BLANC, ROUGE",36.5,https://www.vinted.fr/items/3775343147-dunk-lo...,https://images1.vinted.net/t/01_01d04_8P1JeNMQ...,"BUSIGNY, FRANCE",Women,30.815128,405,374.184872,115.888710,110.888710
3,32.0,Nike dunk,9.0,NIKE,"NOIR, BLANC",24,https://www.vinted.fr/items/3798792190-nike-du...,https://images1.vinted.net/t/01_01adb_QX91JnAe...,"ÉPINAY-SUR-SEINE, FRANCE",Kid,2.203805,405,402.796195,115.888710,106.888710
4,18.0,Nike dunk bambino/a,10.0,NIKE,"NOIR, BLANC",28,https://www.vinted.fr/items/3884037167-nike-du...,https://images1.vinted.net/t/02_00e82_hYs9Zuoc...,"TORINO, ITALIE",Kid,116.904387,405,288.095613,115.888710,105.888710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4080,NaN,NIKE SB Dunk Low en cuir,2677.5,NIKE,Bleu,43.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie,Men,211.591248,405,193.408752,115.243548,-2562.256452
4081,NaN,NIKE SB Dunk Low en cuir,4095.0,NIKE,Rouge,43.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie,Men,211.591248,405,193.408752,115.243548,-3979.756452
4082,NaN,NIKE SB Dunk Low en cuir,4252.5,NIKE,Marron,44.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie,Men,211.591248,405,193.408752,115.243548,-4137.256452
4083,NaN,NIKE SB Dunk Low en cuir,5250.0,NIKE,Marron,40.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,France,Men,50.586158,405,354.413842,115.243548,-5134.756452


In [12]:
df_nikedunk = df_nikedunk.drop(df_nikedunk.columns[0], axis=1)


In [13]:
df_nikedunk.sort_values(by='Price', inplace=True)
df_nikedunk

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender,Approximate Carbon Footprint (kg CO2),Approximative Carbon Print from Ordering on Nike (kg CO2/ton),Carbon Profit (kg CO2),Average price of the product on Nike,Profit Made
0,Dunk low,1.0,NIKE,"BLANC, BLEU","44,5",https://www.vinted.fr/items/3890663184-dunk-lo...,https://images1.vinted.net/t/03_0186f_nVyJSyue...,"GIF-SUR-YVETTE, FRANCE",Men,4.698729,405,400.301271,115.888710,114.888710
1,Stikers autocollants Nike Dunk low Off-White e...,3.5,NIKE,MULTICOLORE,Taille non spécifiée,https://www.vinted.fr/items/3832550323-stikers...,https://images1.vinted.net/t/02_003d4_6uf2Pfz6...,FRANCE,Kid,50.586158,405,354.413842,115.888710,112.388710
2,dunk low,5.0,NIKE,"BLANC, ROUGE",36.5,https://www.vinted.fr/items/3775343147-dunk-lo...,https://images1.vinted.net/t/01_01d04_8P1JeNMQ...,"BUSIGNY, FRANCE",Women,30.815128,405,374.184872,115.888710,110.888710
3,Nike dunk,9.0,NIKE,"NOIR, BLANC",24,https://www.vinted.fr/items/3798792190-nike-du...,https://images1.vinted.net/t/01_01adb_QX91JnAe...,"ÉPINAY-SUR-SEINE, FRANCE",Kid,2.203805,405,402.796195,115.888710,106.888710
4,Nike dunk bambino/a,10.0,NIKE,"NOIR, BLANC",28,https://www.vinted.fr/items/3884037167-nike-du...,https://images1.vinted.net/t/02_00e82_hYs9Zuoc...,"TORINO, ITALIE",Kid,116.904387,405,288.095613,115.888710,105.888710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4080,NIKE SB Dunk Low en cuir,2677.5,NIKE,Bleu,43.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie,Men,211.591248,405,193.408752,115.243548,-2562.256452
4081,NIKE SB Dunk Low en cuir,4095.0,NIKE,Rouge,43.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie,Men,211.591248,405,193.408752,115.243548,-3979.756452
4082,NIKE SB Dunk Low en cuir,4252.5,NIKE,Marron,44.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Italie,Men,211.591248,405,193.408752,115.243548,-4137.256452
4083,NIKE SB Dunk Low en cuir,5250.0,NIKE,Marron,40.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,France,Men,50.586158,405,354.413842,115.243548,-5134.756452


In [14]:
df_nikedunk.to_csv("FINAL_AllData_NikeDunk.csv",index=False)

In [15]:
df_nikemag_vinted = pd.read_csv("AllData_NikeMag.csv",sep=",")
df_nikemagVC =  pd.read_csv("VC_AllData_NikeMag.csv",sep=",")

In [16]:
df_nikemag = pd.concat([df_nikemag_vinted, df_nikemagVC], axis=0, ignore_index=True)
df_nikemag

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender,Approximate Carbon Footprint (kg CO2),Approximative Carbon Print from Ordering on Nike (kg CO2/ton),Carbon Profit (kg CO2),Average price of the product on Nike,Profit Made
0,Nike schoenen,5.00,NIKE,"GRIS, ROSE",36,https://www.vinted.fr/items/487316313-nike-sch...,https://images1.vinted.net/t/01_018a6_YouAt8aC...,"SINT-OEDENRODE, PAYS-BAS",Kid,74.813111,405,330.186889,148.594444,143.594444
1,Nike star runner lichtblauw maat 26,5.00,NIKE,BLEU CLAIR,26,https://www.vinted.fr/items/3759104432-nike-st...,https://images1.vinted.net/t/02_00b26_5A5aoVEL...,"APELDOORN, PAYS-BAS",Kid,89.647343,405,315.352657,148.594444,143.594444
2,Zwarte Nike - maat 38,5.00,NIKE,"NOIR, BLANC",38,https://www.vinted.fr/items/3660145888-zwarte-...,https://images1.vinted.net/t/02_007b4_UkQoacaM...,"HAMME, BELGIQUE",Women,55.744253,405,349.255747,148.594444,143.594444
3,Nike air Max maat 21,5.00,NIKE,"BLANC, VIOLET",21,https://www.vinted.fr/items/3030702278-nike-ai...,https://images1.vinted.net/t/03_00d4f_ry6CtrXX...,PAYS-BAS,Kid,87.928759,405,317.071241,148.594444,143.594444
4,Leuke gympen,5.00,NIKE,"GRIS, MULTICOLORE",38,https://www.vinted.fr/items/2646578334-leuke-g...,https://images1.vinted.net/t/03_00351_JdzuqXb5...,"ROTTERDAM, PAYS-BAS",Kid,74.638487,405,330.361513,148.594444,143.594444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
868,JORDAN En cuir Baskets,141.91,JORDAN,Rouge,9.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Etats-Unis,Women,1522.772797,405,-1117.772797,111.485556,-30.424444
869,JORDAN En cuir Baskets,146.12,JORDAN,Noir,8.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Etats-Unis,Women,1522.772797,405,-1117.772797,111.485556,-34.634444
870,JORDAN En cuir Baskets,146.12,JORDAN,Noir,9.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/images/...,Etats-Unis,Women,1522.772797,405,-1117.772797,111.485556,-34.634444
871,JORDAN En cuir Baskets,156.63,JORDAN,Noir,9.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Etats-Unis,Women,1522.772797,405,-1117.772797,111.485556,-45.144444


In [17]:
df_nikemag.sort_values(by='Price', inplace=True)

In [18]:
df_nikemag.to_csv("FINAL_AllData_NikeMag.csv",index=False)

In [19]:
df_nikeairmax1_vinted = pd.read_csv("AllData_NikeAirmax1.csv",sep=",")
df_nikeairmax1VC =  pd.read_csv("VC_AllData_NikeAirmax1.csv",sep=",")

In [20]:
df_nikeairmax1 = pd.concat([df_nikeairmax1_vinted, df_nikeairmax1VC], axis=0, ignore_index=True)
df_nikeairmax1

,Title,Price,Brand,Color,Size,Link,Image Source,Localisation,Gender,Approximate Carbon Footprint (kg CO2),Approximative Carbon Print from Ordering on Nike (kg CO2/ton),Carbon Profit (kg CO2),Average price of the product on Nike,Profit Made
0,Air max taille 21,8.00,NIKE,"GRIS, JAUNE",21,https://www.vinted.fr/items/3678325504-air-max...,https://images1.vinted.net/t/02_00d84_3Wu7J6o1...,FRANCE,Kid,50.586158,405,354.413842,138.645882,130.645882
1,Basket air max 1,8.00,NIKE,"BLANC, ROSE",29,https://www.vinted.fr/items/3839735280-basket-...,https://images1.vinted.net/t/03_00008_kVVmSUMY...,"ROCHE-LA-MOLIÈRE, FRANCE",Kid,81.726695,405,323.273305,138.645882,130.645882
2,"Basket Nike ""Air Max""",8.00,NIKE,"BLANC, ROSE",35,https://www.vinted.fr/items/3904312378-basket-...,https://images1.vinted.net/t/01_0027d_5fVmJV7V...,"MENNECY, FRANCE",Women,6.561751,405,398.438249,138.645882,130.645882
3,Sapatilhas Nike Air Max tamanho 23 1/2,8.00,NIKE,"GRIS, ROSE",23,https://www.vinted.fr/items/2987733294-sapatil...,https://images1.vinted.net/t/02_011ef_4kBjaDBz...,"LOBÃO, PORTUGAL",Kid,244.273365,405,160.726635,138.645882,130.645882
4,"Obra ""Nike AIR Max _ 1/97 """,8.00,NIKE,"BLANC, MULTICOLORE",44,https://www.vinted.fr/items/3905942860-obra-ni...,https://images1.vinted.net/t/01_014c3_yKAPeDDp...,ESPAGNE,Men,240.819687,405,164.180313,138.645882,130.645882
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2721,NIKE Baskets basses Air Max 1/97 Sean Wothersp...,1482.60,NIKE,Autre,8.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,France,Men,50.586158,405,354.413842,134.999412,-1347.600588
2722,NIKE Air Force 1,1627.50,NIKE,Gris,7.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,France,Men,50.586158,405,354.413842,134.999412,-1492.500588
2723,NIKE Air Force 1 en cuir,1653.75,NIKE,Multicolore,44.0,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,Autriche,Men,176.990280,405,228.009720,134.999412,-1518.750588
2724,NIKE Air Force 1,2097.90,NIKE,Noir,42.5,https://fr.vestiairecollective.com/chaussures-...,https://images.vestiairecollective.com/cdn-cgi...,France,Men,50.586158,405,354.413842,134.999412,-1962.900588


In [21]:
df_nikeairmax1.sort_values(by='Price', inplace=True)

In [22]:
df_nikeairmax1.to_csv("FINAL_AllData_NikeAirmax1.csv",index=False)